<a href="https://colab.research.google.com/github/Antonio-Onyx/Retrieval-Augmented-Generation-RAG-/blob/main/Groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
try:
  !pip install groq
  from groq import Groq
except:
  pass
from google.colab import userdata

# Probando Groq

In [7]:
try:
  groq_api_key = userdata.get("GROQ_API_KEY")
except Exception as e:
  print(f"Error al cargar la clave de Groq {e}")

In [8]:
client = Groq(
    api_key=groq_api_key
)

In [9]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"user",
            "content":"Explica el mecanismo de atencion en los transformers"
        }
    ],
    model="llama3-70b-8192"
)
print(chat_completion.choices[0].message.content)

Un tema fascinante!

En los modelos de Transformers, como el famoso BERT, el mecanismo de atención (Attention Mechanism) es una componente crucial que les permite procesar secuencias de entrada de manera eficiente y enfocarla en las partes más relevantes de la entrada.

**¿Qué es la atención en los Transformers?**

La atención en los Transformers se refiere a la capacidad del modelo de asignar pesos de importancia a diferentes partes de la entrada mientras se procesa. Esto le permite al modelo destacar las características más relevantes de la entrada y enfocar la atención en ellas.

**¿Cómo funciona el mecanismo de atención?**

El mecanismo de atención en los Transformers se basa en tres principales componentes:

1. **Query (Q)**: Representa la consulta o el estado actual del modelo.
2. **Key (K)**: Representa las claves o características importantes de la entrada.
3. **Value (V)**: Representa los valores asociados a las claves.

El mecanismo de atención se puede dividir en tres etapas

# Vamos a empezar con la estructura del RAG

## Retrieval

In [10]:
try:
  !pip install langchain_community
  !pip install pypdf
except:
  pass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 4.9 MB/s eta 0:00:00


### text splitter

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500,
    add_start_index=True,
)

In [13]:
pdf_path = "/content/gauss_fields.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"Documento cargado, numero de paginas {len(documents)}")

Documento cargado, numero de paginas 19


In [14]:
loader

In [15]:
chunks = text_splitter.split_documents(documents)

In [16]:
print(f"Número de chunks: {len(chunks)}")
print("\n Primer chunk:")
print(chunks[0].page_content)

Número de chunks: 19

 Primer chunk:
arXiv:2006.11870v1  [math.NT]  21 Jun 2020
GENUS FIELDS OF KUMMER ℓn–CYCLIC EXTENSIONS
CARLOS DANIEL REYES–MORALES AND GABRIEL VILLA–SAL V ADOR
A B S T R A C T . W e give a construction of the genus ﬁeld for Kummer ℓn-cyclic ex-
tensions of rational congruence function ﬁelds, where ℓ is a prime number . First,
we compute the genus ﬁeld of a ﬁeld contained in a cyclotomic f unction ﬁeld, and
then for the general case. This generalizes the result obtai ned by Peng for a Kum-
mer ℓ-cyclic extension. Finally , we study the extension (K1K2)ge/(K1)ge(K2)ge,
for K1, K2 abelian extensions of k.
1. I NT RODU CT ION
The origin of genus ﬁelds dates back to C. F . Gauss [8] in his wo rk about binary
quadratic forms. For a ﬁnite ﬁeld extension K of Q, the rational number ﬁeld, the
genus ﬁeld Kge is deﬁned as the maximum unramiﬁed ﬁeld extension of K such
that it is the composite Kk∗, where k∗ is an abelian ﬁeld extension over Q. This
deﬁnition is due to A. Fr ¨ 

### Embedding
embedding model from HungginFace

In [17]:
try:
  !pip install -qU langchain-huggingface
except:
  pass

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
vector_1 = embeddings.embed_query(chunks[0].page_content)
vector_2 = embeddings.embed_query(chunks[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 384

[-0.09074479341506958, 0.06299955397844315, 0.06687162071466446, -0.02946328930556774, -0.03094075806438923, 0.011214579455554485, 0.039026226848363876, 0.03662385419011116, 0.05328264832496643, -0.04078393429517746]


### Vector stores

In [20]:
try:
  !pip install faiss-cpu
except:
  pass

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.0 MB/s eta 0:00:00


In [21]:
embedding_dim = len(embeddings.embed_query("Hola Mundo"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(), # los documentos se almacenan en la RAM de la pc
    index_to_docstore_id={}
)

In [22]:
ids = vector_store.add_documents(documents=chunks)

In [23]:
ids

['7c42454b-afa7-4d2d-9063-d98495308dac',
 'c35dffaa-e110-4461-ab17-404e7cff9738',
 '8be43de0-bd71-4cd6-b8ea-7476894fc8b9',
 '8d70b8bb-ac4f-442c-a95d-b3a310f6e496',
 '8fabf6b7-4df3-4ba7-ae8b-1603ec3f675e',
 '7327793d-ae4c-466d-91de-b572ae66be11',
 '2c185a88-9320-4f95-b29f-bb8822ac7a8b',
 'caeaccc4-ccb5-4870-8ef5-02c516956a9b',
 'bebacb35-3bf8-45e4-914d-6ae7148d8cbf',
 'c90d93ef-fc30-4f24-bab7-4c48b8b1b9a2',
 'd21bbdbf-5d79-4e8a-9454-4b4940f5e104',
 '69440275-5ef8-4d38-8619-28df7f52f148',
 '32f03e55-204b-4b97-b8e8-3b2887904555',
 '73c515a6-c26d-456f-88fa-d4b3e52a7a62',
 '3d4871e4-dc74-41e8-a241-e66213c4d363',
 '9cb43912-20a2-4737-9dfc-72b7231fa8a5',
 'f43f4f23-60af-4d9b-95a7-1d93989737ce',
 '35926e00-bdfc-4584-9b1e-47000ecd7b07',
 'cb1cdecc-8a75-44e6-be72-72857fcb50f5']

In [24]:
results = vector_store.similarity_search(
    "genus field"
)

print(results[0])

page_content='GENUS FIELDS OF KUMMER ℓn–CYCLIC EXTENSIONS 19
DE PA RTA M E N T O D E CO N T R O L A U T O M ´A T IC O , C E N T R O D E IN V E S T I G A C I ´O N Y D E E S T U D I O S AVA N-
Z A D O S D E L I. P . N.
E-mail address : mcenigm@gmail.com, creyes@ctrl.cinvestav.mx
DE PA RTA M E N T O D E CO N T R O L A U T O M ´A T IC O , C E N T R O D E IN V E S T I G A C I ´O N Y D E E S T U D I O S AVA N-
Z A D O S D E L I. P . N.
E-mail address : gvillasalvador@gmail.com, gvilla@ctrl.cinvestav.mx' metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2021-11-08T03:55:40-05:00', 'moddate': '2021-11-08T03:55:40-05:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '/content/gauss_fields.pdf', 'total_pages': 19, 'page': 18, 'page_label': '19', 'start_index': 0}


In [25]:
results = await vector_store.asimilarity_search("What does the proposition 2.1. say?")
print(results[0])

page_content='14 C. REYES AND G. VILLA
Proposition 5.3. Let K1, K2 ⊆ nk(Λ N )m, K = K1K2 and E1, E2 as before. If Kge =
(K1)ge(K2)ge, then Ege = ( E1)ge(E2)ge.
Proof. Let K = K1K2 ⊆ nk(Λ N )m and E := E1E2 ⊆ k(Λ N ). Consider the ﬁelds
(Kge)m and (Ege)m. Now , from [4, Theorem 2.2] we have (Ege)m = ( Kge)m and
((Ki)ge)m = (( Ei)ge)m, i = 1 , 2. Therefore
(Ege)m = ( Kge)m = (( K1)ge(K2)ge)m = (( K1)ge)m((K2)ge)m
= (( E1)ge)m((E2)ge)m = (( E1)ge(E2)ge)m.
Finally , by the Galois correspondence, it follows
Ege = ( Ege)m ∩ k(Λ N ) = (( E1)ge(E2)ge)m ∩ k(Λ N ) = ( E1)ge(E2)ge.
□
The converse of Proposition 5.2 does not hold in general, tha t is, if Ege =
(E1)ge(E2)ge, then the equality Kge = ( K1)ge(K2)ge may fail.
Example 5.4. Let K1 = k( ℓ√P1), K2 = k( ℓ√γP2) and K = K1K2 be such that
P1, P2 ∈ R+
T are different polynomials with deg P1 = a, 1 ≤ a < ℓ , deg P2 = ℓ − a
and γ /∈ (F∗
q )ℓ. Then we have Ei = k( ℓ
√
(Pi)∗), i = 1 , 2 and E = E1E2. If χPi is the
Dirichlet character associated wit

## Retrieval & Generations

integracion del mecanismo de recuperacion con Groq

In [26]:
try:
  !pip install -qU langchain-groq
except:
  pass

In [27]:
os.environ["GROQ_API_KEY"] = groq_api_key

In [28]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

### creando el retriever

In [29]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # recuperar los 5 vectores más similares o en otras palabras los 3 documentos más relevantes

### plantilla para el prompt

In [30]:
from langchain_core.prompts import ChatPromptTemplate

In [31]:
template = """
Eres un asistente de investigacion inteligente y traductor simultaneo. Tu tarea es responder a las preguntas del usuario basandote estrictamente en el 'Contexto' proporcionado.

Instrucciones:
1. Lee cuidadosamente la 'pregunta del usuario' y el 'contexto' que se te proporciona. Ambos estaran en ingles.
2. Utiliza únicamente la información que encuentres en el 'Contexto' para formular tu respuesta. No uses conocimiento externo.
3. Si la respuesta puede encontrarse en el 'Contexto', formula una respuesta clara y concisa en **ESPAÑOL**.
4. Si la información necesaria para responder la pregunta no está presente en el 'Contexto', simplemente indica en español: "Lo siento, no tengo suficiente información en los documentos proporcionados para responder a esa pregunta.".
5. Evita inventar información o hacer suposiciones.
6. Manten tus respuestas objetivas y neutrales.

Contexto:
{context}

Pregunta:
{question}

Respuesta:
"""

prompt = ChatPromptTemplate.from_template(template)

In [32]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nEres un asistente de investigacion inteligente y traductor simultaneo. Tu tarea es responder a las preguntas del usuario basandote estrictamente en el \'Contexto\' proporcionado.\n\nInstrucciones:\n1. Lee cuidadosamente la \'pregunta del usuario\' y el \'contexto\' que se te proporciona. Ambos estaran en ingles.\n2. Utiliza únicamente la información que encuentres en el \'Contexto\' para formular tu respuesta. No uses conocimiento externo.\n3. Si la respuesta puede encontrarse en el \'Contexto\', formula una respuesta clara y concisa en **ESPAÑOL**.\n4. Si la información necesaria para responder la pregunta no está presente en el \'Contexto\', simplemente indica en español: "Lo siento, no tengo suficiente información en los documentos proporc

aqui vamos a conectar todas las piezas

In [33]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {'context': retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# echando a andar el RAG

In [34]:
consulta = "Desde donde viene el origen del Genus Field?"
respuesta = rag_chain.invoke(consulta)
print(f"Pregunta: {consulta}")
print(f"Respuesta: {respuesta}")

Pregunta: Desde donde viene el origen del Genus Field?
Respuesta: Según el contexto, el origen del Genus Field se remonta a C. F. Gauss en su trabajo sobre formas cuadráticas binarias.


In [35]:
consulta_2 = "De qué trata el documento Genus field of kummer?"
respuesta_2 = rag_chain.invoke(consulta_2)
print(f"Pregunta: {consulta_2}")
print(f"Respuesta: {respuesta_2}")

Pregunta: De qué trata el documento Genus field of kummer?
Respuesta: El documento "Genus Fields of Kummer ℓn-Cyclic Extensions" trata sobre la construcción del campo de género para extensiones cíclicas de Kummer ℓn de campos de congruencia funcionales racionales. Los autores presentan una construcción del campo de género para campos cíclicos ℓn de Kummer y estudian la relación entre el campo de género y la extensión cíclica. El documento también proporciona resultados generales sobre la estructura del campo de género y su relación con la teoría de la ramificación en campos de funciones.


In [37]:
consulta_3 = "Cual es la documentación que se esta usando en el documento?"
respuesta_3 = rag_chain.invoke(consulta_3)
print(f"Pregunta: {consulta_3}")
print(f"Respuesta: {respuesta_3}")

Pregunta: Cual es la documentación que se esta usando en el documento?
Respuesta: Según el contexto proporcionado, se está utilizando una documentación relacionada con el artículo "Genus Fields of Kummer ℓn-cyclic Extensions" de Carlos Daniel Reyes-Morales y Gabriel Villa-Salvador.


In [38]:
consulta_4 = "en el documento qué es lo que dice la proposition2.1."
respuesta_4 = rag_chain.invoke(consulta_4)
print(f"Pregunta: {consulta_4}")
print(f"Respuesta: {respuesta_4}")

Pregunta: en el documento qué es lo que dice la proposition2.1.
Respuesta: Lo siento, no tengo suficiente información en los documentos proporcionados para responder a esa pregunta. No hay un documento que contenga una "Proposition 2.1" que pueda ser consultada.


In [39]:
consulta_5 = "what does the Notation say us?"
respuesta_5 = rag_chain.invoke(consulta_5)
print(f"Pregunta: {consulta_5}")
print(f"Respuesta: {respuesta_5}")

Pregunta: what does the Notation say us?
Respuesta: La sección de Notación en el contexto proporcionado nos dice que se utilizarán las siguientes notaciones:

* gcd: el mayor común divisor
* lcm: el menor común múltiplo
* νℓ: la valuación respecto a ℓ, un número primo
* k: el campo racional de funciones Fq(T)
* RT: el anillo de polinomios Fq[T]
* R+T: el conjunto de polinomios monicos e irreducibles en RT
* K: una extensión cíclica finita de k
* P∞: el primo infinito en k
* S∞(K): el conjunto de primos en K sobre P∞
* ne∞(E/F): el índice de ramificación de P∞ en la extensión de campos E/F
* nf∞(E/F): el grado de inercia de P∞ en la extensión de campos E/F
* ΛN: el N-torsor del módulo de Carlitz para N ∈ RT \ {0}
* k(ΛN): el campo de funciones cíclotómico sobre k determinado por N
* k(ΛN)+: el subcampo real máximo de k(ΛN)
* Ln: el subcampo de k(ΛT−n−1) fijo por F∗q
* Km: el campo KFqm
* KLn: el campo KLn
* KFqm Ln: el campo KFqm Ln.


In [40]:
consulta_6 = "what is the origin of genus field?"
respuesta_6 = rag_chain.invoke(consulta_6)
print(f"Pregunta: {consulta_6}")
print(f"Respuesta: {respuesta_6}")

Pregunta: what is the origin of genus field?
Respuesta: De acuerdo con el contexto, el origen de los campos de género se remonta al trabajo de C. F. Gauss sobre formas cuadradas binarias. Este concepto fue posteriormente generalizado por A. Fröhlich.


In [42]:
consulta_7 = "what does the M.Rosen's definition say?"
respuesta_7 = rag_chain.invoke(consulta_7)
print(f"Pregunta: {consulta_7}")
print(f"Respuesta: {respuesta_7}")

Pregunta: what does the M.Rosen's definition say?
Respuesta: Según el contexto, la definición de M. Rosen dice que, dado un conjunto finito no vacío S de lugares de un campo de función global K, el campo de clase de Hilbert (relativo a S) KH,S de K es el máximo extension abeliana no ramificada de K tal que los lugares en S se descomponen completamente en KH,S.


# Una pequeña prueba con gradio

In [43]:
!pip install gradio
import gradio as gr

In [44]:
def rag_chat(consulta):
  return rag_chain.invoke(consulta)

In [45]:
interfaz = gr.Interface(fn=rag_chat, inputs="text", outputs="text")
interfaz.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://672d87120f68519506.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## ¡He creado un sistema de RAG con Langchain y Groq! 🚀

Estoy emocionado de compartir mi último proyecto donde he implementado un sistema de Generación Aumentada por Recuperación (RAG) utilizando Langchain y Groq.

Este proyecto me ha permitido explorar cómo combinar la potencia de los Large Language Models (LLMs) con la capacidad de recuperar información relevante de documentos personalizados.

**¿Qué he logrado?**

1.  **Procesamiento de Documentos:** Cargué y dividí un documento PDF técnico en fragmentos manejables.
2.  **Creación de Embeddings:** Convertí los fragmentos de texto en representaciones vectoriales utilizando un modelo de Hugging Face.
3.  **Almacenamiento y Búsqueda Eficiente:** Implementé un vector store (FAISS) para almacenar los embeddings y permitir búsquedas rápidas de información relevante.
4.  **Integración con LLM:** Conecté el sistema de recuperación con un potente modelo de lenguaje de Groq (`llama3-70b-8192`) para generar respuestas coherentes y basadas en el contexto recuperado.
5.  **Construcción de la Cadena RAG:** Organicé todo el flujo utilizando Langchain para crear una cadena RAG robusta.
6.  **Interfaz Interactiva:** Desarrollé una interfaz simple con Gradio para interactuar con el sistema de manera fácil y probar su funcionamiento.

Este proyecto demuestra la capacidad de construir sistemas de IA conversacionales que pueden interactuar con conocimiento específico, abriendo puertas a aplicaciones en áreas como atención al cliente, investigación y análisis de documentos.

¡Estoy aprendiendo y construyendo cosas increíbles! Si te interesa el RAG, Langchain, Groq o los LLMs, ¡conectemos y hablemos!

\#RAG \#Langchain \#Groq \#LLM \#NLP \#InteligenciaArtificial \#MachineLearning \#DeepLearning \#DataScience \#ProyectosAI \#Innovación